In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('D:\projects\LARGE Projects\poems_classification\Poem_classification - train_data.csv')


In [3]:
df = df.dropna(subset=['Poem'])
df.head(10)

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
6,Music,For Frank X WalkerFXW: I don’t know how to sw...
7,Music,My neighbor to the left had a stroke a couple...
8,Music,—for a sixty-seven-pound nugget of Lake Superi...
9,Music,—Issa Rae Everybody Black is my hometown team....
10,Music,"""Save your hands,” my mother says, seeing me u..."


In [4]:
df.head(10)

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
6,Music,For Frank X WalkerFXW: I don’t know how to sw...
7,Music,My neighbor to the left had a stroke a couple...
8,Music,—for a sixty-seven-pound nugget of Lake Superi...
9,Music,—Issa Rae Everybody Black is my hometown team....
10,Music,"""Save your hands,” my mother says, seeing me u..."


### TFIDF Encoding

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3),max_features=10000,stop_words='english',min_df=3,max_df=0.3)
vectorizer.fit(df['Poem'])

x = vectorizer.transform(df['Poem'])

feature_names = vectorizer.get_feature_names_out()
print("length of feature_names:",len(feature_names))
print("length of x:",len(x.toarray()))

final_df = pd.DataFrame(x.toarray(), columns = feature_names)

length of feature_names: 1568
length of x: 837


In [8]:
#sentence transformers encoding

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(df['Poem'].tolist(), show_progress_bar=True)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 27/27 [00:30<00:00,  1.14s/it]


In [20]:
print(len(embeddings))
print(len(embeddings[0]))

837
384


### Train Test SplitZ

In [22]:
from sklearn.model_selection import train_test_split

xtrainem,xtestem,ytrainem,ytestem = train_test_split(embeddings,df['Genre'],test_size=0.2,random_state=42)
xtraintf,xtesttf,ytraintf,ytesttf = train_test_split(x,df['Genre'],test_size=0.2,random_state=42)

### Random forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rfem = RandomForestClassifier(n_estimators=100,random_state=42)
rftf = RandomForestClassifier(n_estimators=100,random_state=42)

rfem.fit(xtrainem,ytrainem)
rftf.fit(xtraintf,ytraintf)

ypredrm = rfem.predict(xtestem)
ypredrf = rftf.predict(xtesttf)

print("Accuracy of sentence transformer model:",accuracy_score(ytestem,ypredrm))
print("Accuracy of tfidf model:",accuracy_score(ytesttf,ypredrf))

print("classification report of sentence transformer model:\n",classification_report(ytestem,ypredrm))
print("classification report of tfidf model:\n",classification_report(ytesttf,ypredrf))

Accuracy of sentence transformer model: 0.49404761904761907
Accuracy of tfidf model: 0.4583333333333333
classification report of sentence transformer model:
               precision    recall  f1-score   support

   Affection       0.47      0.32      0.38        22
       Death       0.41      0.42      0.42        45
 Environment       0.63      0.69      0.65        54
       Music       0.42      0.43      0.42        47

    accuracy                           0.49       168
   macro avg       0.48      0.46      0.47       168
weighted avg       0.49      0.49      0.49       168

classification report of tfidf model:
               precision    recall  f1-score   support

   Affection       0.35      0.27      0.31        22
       Death       0.51      0.44      0.48        45
 Environment       0.58      0.56      0.57        54
       Music       0.35      0.45      0.39        47

    accuracy                           0.46       168
   macro avg       0.45      0.43      0.4

### Logistic Regression model

In [32]:
from sklearn.linear_model import LogisticRegression

lrem = LogisticRegression(max_iter=1000,random_state=42)
lrtf = LogisticRegression(max_iter=1000,random_state=42)

lrem.fit(xtrainem,ytrainem)
lrtf.fit(xtraintf,ytraintf)

ypredlm = lrem.predict(xtestem)
ypredlf = lrtf.predict(xtesttf)

print("Accuracy of sentence transformer model:",accuracy_score(ytestem,ypredlm))
print("Accuracy of tfidf model:",accuracy_score(ytesttf,ypredlf))

print("classification report of sentence transformer model:\n",classification_report(ytestem,ypredlm))
print("classification report of tfidf model:\n",classification_report(ytesttf,ypredlf))



Accuracy of sentence transformer model: 0.6130952380952381
Accuracy of tfidf model: 0.44642857142857145
classification report of sentence transformer model:
               precision    recall  f1-score   support

   Affection       0.39      0.50      0.44        22
       Death       0.71      0.53      0.61        45
 Environment       0.66      0.72      0.69        54
       Music       0.62      0.62      0.62        47

    accuracy                           0.61       168
   macro avg       0.59      0.59      0.59       168
weighted avg       0.63      0.61      0.61       168

classification report of tfidf model:
               precision    recall  f1-score   support

   Affection       0.17      0.09      0.12        22
       Death       0.41      0.40      0.40        45
 Environment       0.57      0.59      0.58        54
       Music       0.41      0.49      0.45        47

    accuracy                           0.45       168
   macro avg       0.39      0.39      0.3

### K-Means Try

In [37]:
final_df.T.nlargest(5,0)

,0,1,2,3,4,5,6,7,8,9,...,827,828,829,830,831,832,833,834,835,836
spend,0.556819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
trickle,0.537236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
mountain,0.429836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
water,0.346378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.150167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.393311
day,0.310794,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
